In [11]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers.legacy import Adam


# 1. Data Loading 


In [3]:
# Load datasets
merged_data = pd.read_csv('/Users/eduardoangeli/Library/CloudStorage/OneDrive-Fanshawec.ca/Capstone/Dataset/merged_data.csv')
merged_data.head()

,home_ownership,annual_inc,loan_amnt,dti,emp_length,label
0,MORTGAGE,55000.0,3600.0,0.1998,10+ years,1
1,MORTGAGE,65000.0,24700.0,0.1998,10+ years,1
2,MORTGAGE,63000.0,20000.0,0.1998,10+ years,1
3,MORTGAGE,110000.0,35000.0,0.1998,10+ years,1
4,MORTGAGE,104433.0,10400.0,0.1998,3 years,1


# 2. Feature Engineering

In [4]:
# One-hot encoding categorical columns ( encoding to categorical variables)
encoded_data = pd.get_dummies(merged_data, columns=['home_ownership', 'emp_length'])

# Splitting data and scaling features (split our data into training and test sets and scale, to normalizing data)
X = encoded_data.drop(columns=['label']) 
y = merged_data['label']

X = X.to_numpy()  # Convert to NumPy array if X is a DataFrame
y = y.to_numpy()  # Convert to NumPy array if y is a DataFrame

# Shuffle the data
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X_shuffled = X[indices]
y_shuffled = y[indices]

# Calculate the split index
test_size = 0.2  # 20% for testing
split_index = int(X.shape[0] * (1 - test_size))

# Split the data
X_train, X_test = X_shuffled[:split_index], X_shuffled[split_index:]
y_train, y_test = y_shuffled[:split_index], y_shuffled[split_index:]

# 3. Model Training and Evaluation

In [7]:
#Convert the Data Types
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [8]:
# Check for NaN values
print(np.isnan(X_train).any(), np.isnan(X_test).any(), np.isnan(y_train).any(), np.isnan(y_test).any())

True True False False


In [9]:
# Replace NaN values if they exist
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

In [14]:
# Model building using TensorFlow
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_shape=(X_train.shape[1],)))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
747737/747737 [==============================] - 252s 336us/step - loss: 14.9898 - accuracy: 0.9553 - val_loss: 15.7047 - val_accuracy: 0.7894
Epoch 2/10
747737/747737 [==============================] - 246s 329us/step - loss: 13.9333 - accuracy: 0.9718 - val_loss: 7.0521 - val_accuracy: 0.9895
Epoch 3/10
747737/747737 [==============================] - 242s 324us/step - loss: 14.6801 - accuracy: 0.9747 - val_loss: 22.9600 - val_accuracy: 0.9838
Epoch 4/10
747737/747737 [==============================] - 244s 326us/step - loss: 15.2465 - accuracy: 0.9762 - val_loss: 10.6507 - val_accuracy: 0.9886
Epoch 5/10
747737/747737 [==============================] - 247s 330us/step - loss: 15.6627 - accuracy: 0.9771 - val_loss: 9.9479 - val_accuracy: 0.9898
Epoch 6/10
747737/747737 [==============================] - 243s 325us/step - loss: 16.0566 - accuracy: 0.9777 - val_loss: 5.3509 - val_accuracy: 0.9930
Epoch 7/10
747737/747737 [==============================] - 242s 324us/step - l

# 4. Return value to the user

In [16]:
def generate_random_user_predict(model, X_train, scaler):
    """Generate a random example of a new user and predict loan acceptance."""
    
    # Randomly select values for each feature
    home_ownership = np.random.choice(['MORTGAGE', 'RENT', 'OWN'])
    annual_inc = np.random.uniform(20000, 150000)  # Random income between 20k and 150k
    loan_amnt = np.random.uniform(500, 40000)  # Random loan amount between $500 and $40k
    dti = np.random.uniform(0.1, 0.9)  # Random dti between 10% and 90%
    emp_length = np.random.choice(['< 1 year', '1 year', '2 years', '3 years', '4 years', '5 years',
                                   '6 years', '7 years', '8 years', '9 years', '10+ years'])
    
    # Creating a DataFrame for the user
    user_data = pd.DataFrame({
        'home_ownership': [home_ownership],
        'annual_inc': [annual_inc],
        'loan_amnt': [loan_amnt],
        'dti': [dti],
        'emp_length': [emp_length]
    })
    
    # One-hot encoding
    user_encoded = pd.get_dummies(user_data, columns=['home_ownership', 'emp_length'])
    
    # Ensuring the user_encoded has all the columns that the model expects
    # If not, they're filled with zeros (because of one-hot encoding)
    missing_cols = set(X_train.columns) - set(user_encoded.columns)
    for col in missing_cols:
        user_encoded[col] = 0
    user_encoded = user_encoded[X_train.columns]
    
    # Scaling the user data
    user_scaled = scaler.transform(user_encoded)
    
    # Making prediction
    prediction = classifier.predict(user_scaled)
    
    # Printing/Returning the result
    if prediction[0] == 1:
        print(f"""Generated user with:
Home Ownership: {home_ownership}
Annual Income: ${annual_inc:.2f}
Loan Amount: ${loan_amnt:.2f}
DTI: {dti*100:.2f}%
Employment Length: {emp_length}

Prediction: Loan ACCEPTED""")
    else:
        print(f"""Generated user with:
Home Ownership: {home_ownership}
Annual Income: ${annual_inc:.2f}
Loan Amount: ${loan_amnt:.2f}
DTI: {dti*100:.2f}%
Employment Length: {emp_length}

Prediction: Loan REJECTED""")

# Calling the function
generate_random_user_predict(model, X_train, scaler)

NameError: name 'scaler' is not defined